# 라이브러리 import

In [1]:
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
import re
from tqdm import tqdm

# 함수 정의

In [18]:
def get_url_list():
    with open("../data/job_fix_list.csv", encoding='cp949') as f:
        urls = f.readlines()

#     url_list = []

#     for url in urls:
#         url_spl = url.rstrip("\n").split(",")

#         job = ",".join(url_spl[:-1]).strip('""')
#         url = url_spl[-1]

#         url_list.append([job, url])

    url_list = [url.rstrip("\n") for url in urls]
    
    return url_list

def get_job_description(driver, url):
    
    # Job Description 을 담을 딕셔너리 생성
    res = {}
    
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            
            # url 접속하기
            driver.get(url)

            # 화면 비율을 축소해서 모든 데이터 보이게 만들기
            driver.execute_script("document.body.style.zoom='10%'")

            time.sleep(5)

            # url 에서 index 값 추출
            res['id'] = url.split("/")[-1]

            # JobHeader 클래스 가져와서 title, company, tags 추출
            job_header = driver.find_element(By.CLASS_NAME, 'JobHeader_className__HttDA')
            title = job_header.find_element(By.TAG_NAME, 'h2')
            company = job_header.find_element(By.TAG_NAME, 'a')
            location = job_header.find_element(By.CLASS_NAME, 'JobHeader_pcLocationContainer__xRwIv')
            tags = job_header.find_element(By.CLASS_NAME, 'Tags_tagsClass__mvehZ')

            # res 딕셔너리에 저장
            res['title'] = title.text
            res['company'] = company.text
            res['location'] = location.text.split(".")

            # Tag 리스트 가져와서 저장
            tag_list = list(map(str, tags.text.split('#')))
            tag_list = tag_list[1:]
            res['tags'] = tag_list

            # JobDescription 클래스 가져와서 header, contents 추출
            jd = driver.find_element(By.CLASS_NAME, 'JobDescription_JobDescription__VWfcb')
            jd_header = jd.find_elements(By.TAG_NAME, 'h6')
            jd_contents = jd.find_elements(By.TAG_NAME, 'p')

            for i in range(1, len(jd_header)+1):
                title = jd_header[i-1].text
                content = jd_contents[i].text.split("\n")

                if title == '자격요건':
                    res['qualifications'] = content
                elif title == '우대사항':
                    res['preferred'] = content
                elif title == '혜택 및 복지':
                    res["benefits"] = content
                elif title == '기술스택 ・ 툴':
                    res["skills"] = content
                else:
                    continue


            jwp = driver.find_element(By.CLASS_NAME, "JobWorkPlace_className__ra6rp")
            jwp_bodies = jwp.find_elements(By.CLASS_NAME, 'body')

            if len(jwp_bodies) < 2:
                workplace = ""
            else:
                workplace = jwp_bodies[1].text

            res['workplace'] = workplace
        except NoSuchElementException:
            print("존재하지 않는 페이지 입니다.")
    else:
        print("존재하지 않는 페이지 입니다.")
        print("res = ", res)
    
    return res

In [19]:
def get_job_description_exception(driver, url):
    
    # Job Description 을 담을 딕셔너리 생성
    res = {}
    
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            
            # url 접속하기
            driver.get(url)

            # 화면 비율을 축소해서 모든 데이터 보이게 만들기
            driver.execute_script("document.body.style.zoom='10%'")

            time.sleep(5)

            # url 에서 index 값 추출
            res['id'] = url.split("/")[-1]

            # JobHeader 클래스 가져와서 title, company, tags 추출
            job_header = driver.find_element(By.CLASS_NAME, 'JobHeader_className__HttDA')
            title = job_header.find_element(By.TAG_NAME, 'h2')
            company = job_header.find_element(By.TAG_NAME, 'a')
            location = job_header.find_element(By.CLASS_NAME, 'JobHeader_pcLocationContainer__xRwIv')
            tags = job_header.find_element(By.CLASS_NAME, 'Tags_tagsClass__mvehZ')

            # res 딕셔너리에 저장
            res['title'] = title.text
            res['company'] = company.text
            res['location'] = location.text.split(".")

            # Tag 리스트 가져와서 저장
            tag_list = list(map(str, tags.text.split('#')))
            tag_list = tag_list[1:]
            res['tags'] = tag_list

            # JobDescription 클래스 가져와서 header, contents 추출
            jd = driver.find_element(By.CLASS_NAME, 'JobDescription_JobDescription__VWfcb')
            jd_header = jd.find_elements(By.TAG_NAME, 'h6')
            jd_contents = jd.find_elements(By.TAG_NAME, 'p')

            for i in range(len(jd_header)):
                title = jd_header[i].text
                content = jd_contents[i].text.split("\n")

                if title == '자격요건':
                    res['qualifications'] = content
                elif title == '우대사항':
                    res['preferred'] = content
                elif title == '혜택 및 복지':
                    res["benefits"] = content
                elif title == '기술스택 ・ 툴':
                    res["skills"] = content
                else:
                    continue


            jwp = driver.find_element(By.CLASS_NAME, "JobWorkPlace_className__ra6rp")
            jwp_bodies = jwp.find_elements(By.CLASS_NAME, 'body')

            if len(jwp_bodies) < 2:
                workplace = ""
            else:
                workplace = jwp_bodies[1].text

            res['workplace'] = workplace
        except NoSuchElementException:
            print("존재하지 않는 페이지 입니다.")
    else:
        print("존재하지 않는 페이지 입니다.")
        print("res = ", res)
    
    return res

# 크롤링 실행

In [21]:
driver = webdriver.Chrome()

# url 가져오기
url_list = get_url_list()

total = []

for i in tqdm(range(1723, 2000)):
    if i == 1723:
        res = get_job_description_exception(driver, url_list[i])
    else:
        res = get_job_description(driver, url_list[i])
    
    if len(res) == 0:
        print("존재하지 않는 페이지 입니다.")
        continue
    else:
        total.append(res)

driver.quit()

  0%|          | 0/277 [00:02<?, ?it/s]


KeyboardInterrupt: 

# 크롤링 결과 및 확인

In [7]:
print(total[0])

for t in total:
    print(t)

{'id': '134267', 'title': '시니어 컴퓨터 비전 엔지니어', 'company': '리디(RIDI)', 'location': ['서울', '한국'], 'tags': ['연봉업계평균이상', '누적투자100억이상', '인원급성장', '301~1,000명', '설립10년이상', '시리얼', '식비', '음료', 'IT, 컨텐츠'], 'qualifications': ['• 컴퓨터 공학, 통계학 등 머신 러닝 관련 분야를 전공하신 분', '• 만 6년 이상의 데이터 분석 및 머신러닝 관련 경력을 보유하신 분', '• 능숙한 실험 설계와 분석이 가능한 분', '• GAN 등 이미지 처리 관련 모델을 다뤄보신 분', '• Neural Style Transfer 등 이미지 생성 경험이 있으신 분'], 'preferred': ['• ML 분야 박사 이상 또는 이에 준하는 경력을 보유하신 분', '• 대규모 데이터를 활용한 모델링 경험이 있는 분', '• 모델의 실제 서비스 적용 및 운영 경험이 있는 분', '• AWS 등 클라우드 서비스 사용 경험', '• TensorFlow, PyTorch 등의 ML 관련 라이브러리 사용 경험', '• Docker, Airflow 등의 운영 관련 도구 사용 경험', '• Spark 등의 데이터 관련 라이브러리 사용 경험'], 'benefits': ['언론에 소개된 리디', "•리디, GIC등 1200억원 투자 유치 및 기업가치 1.6조 '유니콘' 기업 등극", 'https://n.news.naver.com/article/277/0005051077', '•웹툰 기업으로 변신한 리디, 女心 공략하며 무섭게 성장', 'https://n.news.naver.com/mnews/article/366/0000826817?sid=101', '•구글 엔지니어 출신 CTO 영입', 'https://www.venturesquare.net/853258', '', '[리디 LIFE: 혜택 및 복지] ', '', '■ WORK', '리더스(Ridu

# 결과 csv 파일로 저장하기

In [8]:
import csv

with open('/Users/jhnam/workspace/playground-data/crawler/output/wanted_crawling_output_4_1501_1723.csv', 'w', encoding='UTF-8') as f:
    # csv 파일의 헤더 설정
    header = ['id', 'title', 'company', 'location', 'tags', 'qualifications', 'preferred', 'benefits', 'skills', 'workplace']

    # 헤더 적용
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    
    # 헤더에 맞게 딕셔너리 값 입력
    for jd in total:
        writer.writerow(jd)

# 전처리 (진행중)

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


exception_list = ['를', '를', '를', '을', '을', '있거나', '있으신', '갖추신', '경험이', '이상의', '에', '에', '에', '에', '에', '이용한', '에', '에', '에', '준하는', '에', '있으신', '에', '에', '에', '중심적인', '에', '에', '이해를', '에', '대한', '및', '가지고', '전반적인', '에']

# 텍스트 정의
contents = "• 6년 이상의 백엔드 개발 경력이 있거나 그에 준하는 실력을 갖추신 분\n• JavaScript / TypeScript 를 이용한 개발에 능숙하신 분\n• 웹 서비스에 대한 전반적인 이해를 갖추신 분\n• RDB(Postgres) 및 ORM에 대한 전반적인 이해를 갖추신 분\n• 도커, 쿠버네티스 등 컨테이너 기반의 서비스에 대한 경험이 있으신 분\n• 1년 이상의 NestJS 실무 프로젝트 개발 경험이 있으신 분\n• 모듈화, 테스트, 유지보수가 용이한 코드에 대한 이해를 갖추고 계신 분\n• 기술 중심이 아닌 유저 중심적인 태도를 가지고 계신 분\n• 배움에 대한 열정을 갖고, 지속적이고 효과적으로 피드백을 주고 받는 분"

# 특수문자 제거
contents = re.sub(r'[^\w\s]', '', contents)

# 단어 토큰화
tokens = word_tokenize(contents)

# 품사 태깅
tagged_words = pos_tag(tokens)

# 명사 추출 (동사 및 불용어 제외)
nouns = [word for word, pos in tagged_words if pos.startswith('N') and not pos.startswith('V') and word.lower() not in exception_list]

# 중복 제거
nouns = list(set(nouns))

# 결과 출력
print(nouns)